<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/Kyle_Large_Dataset_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Jupyter notebook is intended to assist with accessing plotly from outside of a Google Colab environment (Should technically work for any cloud environment that allows you to install NGROK). The problem with running plotly within Google Colab is that the ports that are opened by Plotly are not exposed to the outside world. In order to expose them we are going to create a network tunnel, that's where NGROK comes in.

A few things of note:
*  If you are using Google Drive for any information for this plot, you will want to copy that data "locally" before the graph will fuction externally. This is because we are starting up a new thread that will not carry over the Google Driver permissions you may have granted to the current thread.


# Download and install NGROK

First thing we need to do is download and install NGROK.

Important thing to note, if you ever get "OSError: [Errno 98] Address already in use" that means there is an instance of dash still running, the simplest solution is to restart the runtime.

In [9]:
# How to run a Dash app in Google Colab

## Requirements

### Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip


get_ipython().system_raw('./ngrok http 8050 &')

### Install Dash
!pip install dash  # The core dash backend
!pip install jupyter_dash

#Do not install any of these. When you install Dash, it includes these. If you call the install it installs a previous version of these and will generate an error.
#!pip install dash-html-components  # HTML components
#!pip install dash-core-components  # Supercharged components
#!pip install dash-table  # Interactive DataTable component (new!)

--2021-07-20 11:35:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.38.23, 54.84.56.8, 52.202.35.83, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.38.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.04MB/s    in 2.2s    

2021-07-20 11:35:18 (6.04 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
     |████████████████████████████████| 1.1 MB 4.3 MB/s 
     |████████████████████████████████| 3.7 MB 17.2 MB/s 
     |████████████████████████████████| 83 kB 2.5 MB/s 
     |████████████████████████████████| 1.8 MB 48.4 MB/s 
     |████████████████████████████████| 357 kB 47.6 MB/s 
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl siz

Now that we have NGROK configured, we need a way to figure out what the URL will be to access NGROK. To do this we are going to have to execute some Linux commands to pull the established tunnels.

Additional Details:

Durring testing we discovered that occassionally the Curl command would come back with nothing. we suspect this is due to the async nature of the command pulling the list of tunnels before any of them have been established. So, we have built in a wait and a couple loops to ensure that we can get a proper URL.

Additionally, every time you restart the app you will need to generate a new tunnel. So make sure this method gets called before you execute your python file to start the server.

In [ ]:
import os
import json
import time


def GetPublicIP(NumberOfLoops):
  NumberOfLoops = NumberOfLoops + 1
  get_ipython().system_raw('./ngrok http 8050 &')
  time.sleep(1)
  result = os.popen("curl -s http://localhost:4040/api/tunnels").read()
  resultDict = json.loads(result)
  if 'tunnels' in resultDict.keys():
    publicIP = resultDict['tunnels'][0]['public_url']
    print(publicIP)
    return publicIP
  else:
    print("Number of Loops: " + NumberOfLoops)
    if NumberOfLoops > 5:
      return "Unable to get public IP"
    return GetPublicIP(NumberOfLoops)


# Connect to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path

GoogleDriveBase = "/content/drive/My Drive/" #Kyle
WorkingDirectory = GoogleDriveBase + "Projects/ACLU/" #Kyle
#GoogleDriveBase = "/content/drive/MyDrive/Shared with me/content/drive/My Drive/Shared with me/" #Anupriya
#WorkingDirectory = GoogleDriveBase + 'ACLU/' #Anupriya

WorkingFiles = WorkingDirectory + 'WorkingFiles/'
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"


#Make the necessary folders for the script to run.
ListOfAllRequiredDirectories = [WorkingDirectory + 'Pickle', 
                          WorkingDirectory + 'WorkingFiles',
                          WorkingDirectory + 'AdditionalData',
                          WorkingDirectory + 'ACLUData']

for folder in ListOfAllRequiredDirectories:
  RunningPath = GoogleDriveBase + folder + "/"
  Path(RunningPath).mkdir(parents=True, exist_ok=True)


If you would like to run the previous example externally, just copy and paste your code into a new block like below. 

In [3]:
import pandas as pd

df = pd.read_csv(WorkingFiles + "dfVoters.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,28,53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Unnamed: 0,index,Voter File VANID,FirstName,MiddleName,LastName,Address,DWID,Zip5,Age,Primary19,MaritalStatus,mCity,Mass_Incarceration,StreetType,StreetSuffix,mAddress,mState,mZip5,mZip4,Sex,City,State,Zip4,Suffix,CD,SD,HD,PreferredEmail,Preferred Phone,Cell Phone,CountyName,DOB,DateReg,Home Phone,EthnicCatalistName,Party,PersonalEmail,RaceName,StreetNo,StreetNoHalf,StreetPrefix,StreetName,AptType,AptNo,City.1,State.1,Zip5.1,Zip4.1,2020_Biden_Support,Voting_Aug_Prim,PoliceAccountability,VBM_Application,MarijuanaConviction,2020_Biden_Support.1,Absentee_Voting
0,0,284,67293,A,C,Anderson,15716 Veronica Ave,17358713,48021,76,NaN,S,Eastpointe,NaN,Ave,NaN,15716 Veronica Ave,MI,48021.0,3639.0,M,Eastpointe,MI,3639.0,NaN,9,9.0,18.0,NaN,"=""3138021223""","=""3138021223""",Macomb,01/14/1945,10/10/2000,"=""5868597737""",Uncoded,U,NaN,Caucasian,15716.0,NaN,NaN,Veronica,NaN,NaN,Eastpointe,MI,"=""48021""","=""3639""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7738,805732,A,Siemen,Schavedebra,30739 Roan Dr,18103616,48093,64,NaN,S,Warren,NaN,Dr,NaN,30739 Roan Dr,MI,48093.0,5620.0,F,Warren,MI,5620.0,NaN,9,9.0,25.0,NaN,"=""5869439289""","=""5869439289""",Macomb,11/01/1956,01/01/1987,NaN,Polish,U,NaN,Caucasian,30739.0,NaN,NaN,Roan,NaN,NaN,Warren,MI,"=""48093""","=""5620""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,128646,17393956,A Hee,NaN,Kim,3201 Burton St SE SE115,931003922,49546,26,NaN,NaN,Grand Rapids,NaN,St,SE,3201 Burton St SE SE115,MI,49546.0,4301.0,F,Grand Rapids,MI,4301.0,NaN,3,29.0,76.0,NaN,NaN,NaN,Kent,01/01/1995,12/19/2015,NaN,Korean,U,NaN,Asian,3201.0,NaN,NaN,Burton,NaN,SE115,Grand Rapids,MI,"=""49546""","=""4301""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,149281,17576115,A Moni,Elanya Danielle,Collier,2578 Hearthside Dr,938316098,48198,24,NaN,NaN,Ypsilanti,NaN,Dr,NaN,2578 Hearthside Dr,MI,48198.0,9286.0,F,Ypsilanti,MI,9286.0,NaN,12,18.0,54.0,NaN,"=""7348831091""","=""7348831091""",Washtenaw,01/01/1997,03/08/2016,NaN,Uncoded,U,NaN,Black,2578.0,NaN,NaN,Hearthside,NaN,NaN,Ypsilanti,MI,"=""48198""","=""9286""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,105835,15387665,A S M,Shahadat,Hossain,12000 Charest St,530769287,48212,42,P,S,Hamtramck,NaN,St,NaN,12000 Charest St,MI,48212.0,2726.0,M,Hamtramck,MI,2726.0,NaN,14,2.0,4.0,NaN,"=""3138937031""","=""3134003535""",Wayne,10/01/1978,09/28/2012,"=""3138937031""",Bangladeshi,U,NaN,Asian,12000.0,NaN,NaN,Charest,NaN,NaN,Hamtramck,MI,"=""48212""","=""2726""",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
#Set up all our filter columns
FilterColumnNames = ["RaceName", "Sex", "Age", "CountyName"]
FilterColumns = {}

for column in FilterColumnNames:
  FilterColumns[column] = list(df[column].unique())
  FilterColumns[column].append("All")
  print(FilterColumns[column])

ImportantColumnNames = FilterColumnNames.copy()
ImportantColumnNames.append('2020_Biden_Support')

df_aggregate = df.groupby(by=ImportantColumnNames).count()
df_aggregate = df_aggregate["index"].reset_index()
df_aggregate = df_aggregate.rename(columns={"index": "Count"})

['Caucasian', 'Asian', 'Black', 'Unknown', 'Hispanic', 'Native American', 'All']
['M', 'F', 'U', 'All']
[76, 64, 26, 24, 42, 46, 65, 35, 22, 28, 25, 27, 62, 53, 29, 33, 38, 32, 30, 43, 36, 49, 21, 51, 50, 41, 20, 37, 47, 45, 58, 39, 40, 31, 55, 34, 96, 69, 52, 44, 60, 57, 48, 23, 63, 56, 19, 81, 59, 73, 79, 70, 68, 67, 78, 77, 71, 54, 66, 61, 80, 74, 72, 84, 94, 18, 91, 75, 93, 103, 89, 97, 98, 105, 92, 87, 86, 83, 82, 85, 90, 88, 95, 99, 104, 100, 108, 102, 101, 112, 'All']
['Macomb', 'Kent', 'Washtenaw', 'Wayne', 'Genesee', 'Ingham', 'Saginaw', 'Oakland', 'Calhoun', 'Kalamazoo', 'Isabella', 'Livingston', 'Ottawa', 'St. Joseph', 'Muskegon', 'Midland', 'Allegan', 'Emmet', 'St. Clair', 'Berrien', 'Marquette', 'Bay', 'Alpena', 'Eaton', 'Monroe', 'Shiawassee', 'Jackson', 'Van Buren', 'Mackinac', 'Lapeer', 'Osceola', 'Clinton', 'Menominee', 'Presque Isle', 'Wexford', 'Houghton', 'Montmorency', 'Lenawee', 'Grand Traverse', 'Dickinson', 'Gratiot', 'Kalkaska', 'Iosco', 'Antrim', 'Montcalm', '

In [118]:
#Pre aggregate the data as much as possible. Even if this only reduces your data by half it will provide a massive improvement to performance
print(str(len(df)) + " records reduce to " + str(len(df_aggregate)))
df_aggregate.head()

149854 records reduce to 4451


,RaceName,Sex,Age,CountyName,2020_Biden_Support,Count
0,Asian,F,21,Washtenaw,1. Strong Support,1
1,Asian,F,22,Wayne,1. Strong Support,1
2,Asian,F,22,Wayne,Not Voting,1
3,Asian,F,23,Macomb,5. Strong Oppose,1
4,Asian,F,23,Wayne,1. Strong Support,2


In [127]:
import altair as alt
import io
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
import plotly.express as px
from vega_datasets import data
#alt.data_transformers.disable_max_rows()

# Don't need this with the cars dataset
alt.data_transformers.enable('default', max_rows=10000)

cars = data.cars()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.css.append_css({'external_url':
                    'https://cdn.rawgit.com/gschivley/8040fc3c7e11d2a4e7f0589ffc829a02/raw/fe763af6be3fc79eca341b04cd641124de6f6f0d/dash.css'
                    })
app.title = 'Test dash and altair'
server = app.server

#Build our dropdowns for each of the Filter Columns
DropDowns = []
inputs = []
for ColumnName in FilterColumnNames:
    DropDowns.append(
        html.Div([
            html.Label(ColumnName),
            dcc.Dropdown(
                id=ColumnName,
                options=[{'label': i, 'value':i} for i in FilterColumns[ColumnName]],
                value='All'
            )
        ],
        style={'width': '250px', 'marginRight': 'auto',
              'marginLeft': 'auto', 'textAlign': 'center'},
        className='column'))
    inputs.append(dash.dependencies.Input(ColumnName, 'value'))

#Build our main layout
app.layout = html.Div(children=[html.Div(children=DropDowns, className='row'), html.Pre(id='plot', children="If this doesn't go away you have an error")])

@app.callback(
    dash.dependencies.Output('plot', 'children'),
    inputs
)
def pick_figure(Race, Sex, Age, CountyName):
    #This is our key. Filter everything before passing it into the charts.
    dfFiltered = df_aggregate[((df_aggregate['RaceName'] == Race) | (Race == 'All')) & 
                              ((df_aggregate['Sex'] == Sex) | (Sex == 'All')) &
                              ((df_aggregate['Age'] == Age) | (Age == 'All')) &
                              ((df_aggregate['CountyName'] == CountyName) | (CountyName == 'All'))]
                              
    #We should also aggregate it. This will keep our record count low
    dfFiltered = dfFiltered.groupby(by=['2020_Biden_Support', 'CountyName']).sum()
    dfFiltered = dfFiltered.reset_index()

    #Encode the Altair Chart
    points = alt.Chart(dfFiltered).mark_bar().encode(
            x=alt.X('CountyName:N'),
            y='total:Q',
            tooltip="Title:N"
        ).transform_aggregate(
        total='sum(Count)',
        groupby=["CountyName"]
    )
    # Save html as a StringIO object in memory
    htmloutput = io.StringIO()
    points.save(htmloutput, 'html')

    chart1 = html.Iframe(
        height='500',
        width='1000',
        #sandbox='allow-scripts',

        # This is where we will pass the html
        srcDoc = htmloutput.getvalue(),

        # Get rid of the border box
        style={'border-width': '0px'}
    )

    #Create a simple Plotly Express chart
    chart2 = dcc.Graph(
      id='example-graph',
      figure=px.histogram(dfFiltered, x="2020_Biden_Support", y="Count", barmode='group', histfunc='sum')
    )

    # Return the html from StringIO object
    return chart2, chart1
    #return chart2

#We can only have 1 instance of Dash running at a time.
app.run_server(mode="inline")

/usr/local/lib/python3.7/dist-packages/dash/resources.py:72: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://cdn.rawgit.com/gschivley/8040fc3c7e11d2a4e7f0589ffc829a02/raw/fe763af6be3fc79eca341b04cd641124de6f6f0d/dash.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources



<IPython.core.display.Javascript object>